## Creating Neural machine translation model to translate english to french (Character level)

---



In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving fra.txt to fra.txt
User uploaded file "fra.txt" with length 10670848 bytes


In [0]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [30]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

#read the data and convert it into utf-8 format to convert french characters

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
lines[:5]

['Go.\tVa !',
 'Run!\tCours\u202f!',
 'Run!\tCourez\u202f!',
 'Wow!\tÇa alors\u202f!',
 'Fire!\tAu feu !']

In [0]:
# We take a 10k sample from original 152584 dataset
for line in lines[: 10000]:
    # Input and target are tab separated so split and text input_text,target_text
    input_text, target_text = line.split('\t')
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    
    # Then we append input_text to input_texts list
    # We do the same with target_text
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # Here we get unique characters in whole text corpus for input as well
    # as for targets
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [34]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [0]:
# Creating lookup table
# Converts input and target characters into dict (key,value) pair where character is the key and value is integer associated with the character
# Like {'A':1}
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


In [56]:
print("encoder_input_data shape (rows,max_seq_len,n.o of unique chars)",encoder_input_data.shape)
print("decoder_input_data shape (rows,max_seq_len,n.o of unique chars)",decoder_input_data.shape)
print("decoder_target_data shape (rows,max_seq_len,n.o of unique chars)",decoder_target_data.shape)

encoder_input_data shape (rows,max_seq_len,n.o of unique chars) (10000, 16, 71)
decoder_input_data shape (rows,max_seq_len,n.o of unique chars) (10000, 59, 94)
decoder_target_data shape (rows,max_seq_len,n.o of unique chars) (10000, 59, 94)


1) encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.

2) decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containg a one-hot vectorization of the French sentences.

3) decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].

In [0]:
# Code below one-hot encodes the data and stores in the np.zero arrays which we create before
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            
            # Here we use t-1 because the t starts with 1(In decoder_target_data) and not zero so you
            # end up putting second char in array into t-1 i.e zero place.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [85]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 335872      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  359424      input_4[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [83]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.9185 - val_loss: 0.9568
Epoch 2/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.7215 - val_loss: 0.7922
Epoch 3/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.6142 - val_loss: 0.6991
Epoch 4/100
1856/8000 [=====>........................] - ETA: 15s - loss: 0.5702

8000/8000 [==============================] - 22s 3ms/step - loss: 0.5568 - val_loss: 0.6509
Epoch 5/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.5215 - val_loss: 0.6161
Epoch 6/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.4883 - val_loss: 0.5886
Epoch 7/100
6272/8000 [======================>.......] - ETA: 4s - loss: 0.4649

8000/8000 [==============================] - 22s 3ms/step - loss: 0.4626 - val_loss: 0.5698
Epoch 8/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.4401 - val_loss: 0.5535
Epoch 9/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.4200 - val_loss: 0.5327
Epoch 10/100
7040/8000 [=========================>....] - ETA: 2s - loss: 0.4046

8000/8000 [==============================] - 22s 3ms/step - loss: 0.4040 - val_loss: 0.5202
Epoch 11/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.3882 - val_loss: 0.5100
Epoch 12/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.3733 - val_loss: 0.4995
Epoch 13/100
7104/8000 [=========================>....] - ETA: 2s - loss: 0.3603

8000/8000 [==============================] - 23s 3ms/step - loss: 0.3602 - val_loss: 0.4898
Epoch 14/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.3476 - val_loss: 0.4919
Epoch 15/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.3357 - val_loss: 0.4839
Epoch 16/100
7104/8000 [=========================>....] - ETA: 2s - loss: 0.3243

...

8000/8000 [==============================] - 23s 3ms/step - loss: 0.0630 - val_loss: 0.7406
Epoch 89/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0622 - val_loss: 0.7318
Epoch 90/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.0609 - val_loss: 0.7441
Epoch 91/100
7104/8000 [=========================>....] - ETA: 2s - loss: 0.0600

8000/8000 [==============================] - 23s 3ms/step - loss: 0.0605 - val_loss: 0.7477
Epoch 92/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0594 - val_loss: 0.7589
Epoch 93/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0587 - val_loss: 0.7582
Epoch 94/100
7104/8000 [=========================>....] - ETA: 2s - loss: 0.0575

8000/8000 [==============================] - 22s 3ms/step - loss: 0.0580 - val_loss: 0.7651
Epoch 95/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0572 - val_loss: 0.7633
Epoch 96/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0562 - val_loss: 0.7634
Epoch 97/100
7104/8000 [=========================>....] - ETA: 2s - loss: 0.0552

8000/8000 [==============================] - 22s 3ms/step - loss: 0.0555 - val_loss: 0.7683
Epoch 98/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0551 - val_loss: 0.7758
Epoch 99/100
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0539 - val_loss: 0.7686
Epoch 100/100
7104/8000 [=========================>....] - ETA: 2s - loss: 0.0529

8000/8000 [==============================] - 22s 3ms/step - loss: 0.0532 - val_loss: 0.7716


Saving model weights to h5 file

In [84]:
model.save('seq2seq.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Downloading model from colab

In [0]:
from google.colab import files

files.download('seq2seq.h5')

In [89]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('True target sentence:',target_texts[seq_index])

-
Input sentence: Go.
Decoded sentence: Va !

True target sentence: 	Va !

-
Input sentence: Run!
Decoded sentence: Courez !

True target sentence: 	Cours !

-
Input sentence: Run!
Decoded sentence: Courez !

True target sentence: 	Courez !

-
Input sentence: Wow!
Decoded sentence: Ça alors !

True target sentence: 	Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

True target sentence: 	Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

True target sentence: 	À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

True target sentence: 	Saute.

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

True target sentence: 	Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

True target sentence: 	Stop !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

True target sentence: 	Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attends !

True target sentence: 	Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

True t

-
Input sentence: Beat it.
Decoded sentence: Dégage !

True target sentence: 	Dégage !

-
Input sentence: Call me.
Decoded sentence: Appelle-moi !

True target sentence: 	Appelle-moi !

-
Input sentence: Call me.
Decoded sentence: Appelle-moi !

True target sentence: 	Appellez-moi !

-
Input sentence: Call us.
Decoded sentence: Appelle-nous !

True target sentence: 	Appelle-nous !

-
Input sentence: Call us.
Decoded sentence: Appelle-nous !

True target sentence: 	Appelez-nous !

-
Input sentence: Come in.
Decoded sentence: Entre.

True target sentence: 	Entrez !

-
Input sentence: Come in.
Decoded sentence: Entre.

True target sentence: 	Entre.

-
Input sentence: Come in.
Decoded sentence: Entre.

True target sentence: 	Entre !

-
Input sentence: Come in.
Decoded sentence: Entre.

True target sentence: 	Entrez !

-
Input sentence: Come on!
Decoded sentence: Allez !

True target sentence: 	Allez !

-
Input sentence: Come on.
Decoded sentence: Entre.

True target sentence: 	Allez !

-
I